In [1]:
import torch
from config import batch_size, block_size, max_iters, eval_interval, learning_rate, eval_iters, n_embd, n_head, n_layer, dropout, sparsity, low_rank
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from loader_data import ShakespeareDataModule

torch.manual_seed(1337)



In [2]:
from transformer.dbt_model import EBTransformerModel
from transformer.dbts_model import EBSTransformerModel

data_module = ShakespeareDataModule(data_dir="data/input.txt", batch_size=batch_size, block_size=block_size)
model = EBTransformerModel(data_module.vocab_size, n_embd, block_size, n_head, dropout, n_layer, learning_rate)
s_model = EBSTransformerModel(data_module.vocab_size, n_embd, block_size, n_head, dropout, n_layer, learning_rate, sparsity, low_rank)


In [3]:

num_epochs = 25

logger = TensorBoardLogger("tb_logs/curr", name=f"dev_{sparsity}")

trainer = pl.Trainer(
    logger=logger,
    accelerator="gpu", 
    limit_train_batches = 500, 
    limit_val_batches = 200,
    max_epochs=num_epochs, 
    precision="16-mixed",
    # overfit_batches=2 
)

trainer.fit(s_model, data_module)
# trainer.fit(model, data_module)



Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                     | Type             | Params
--------------------------------------------------------------
0 | token_embedding_table    | Embedding        | 25.0 K
1 | position_embedding_table | Embedding        | 98.3 K
2 | blocks                   | Sequential       | 10.9 M
3 | ln_f                     | LayerNorm        | 768   
4 | lm_head                  | Linear         

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/maciej/anaconda3/envs/ml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/maciej/anaconda3/envs/ml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


/home/maciej/anaconda3/envs/ml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 1:  13%|█▎        | 67/500 [00:17<01:50,  3.91it/s, v_num=7, train_loss=2.480] 

/home/maciej/anaconda3/envs/ml/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
torch.no_grad()